In [1]:
from svfpy import svfpy
import numpy as np
import geopandas as gpd
import pandas as pd
import rioxarray

In [2]:
mds = '/media/fernando/DATA/nyc_lidar/results/DSM/all-DSM-NYC.tiff'
svf_file = '/media/fernando/DATA/svf-nyc/svf_all.tiff'

In [3]:
svf = svfpy.SVF(
    mds, '/media/fernando/DATA/svf-nyc/', 
    multiplier=0.3048,
    kernel_size_side=800, 
    tmp_folder='/media/fernando/DATA/svf-nyc/', 
    max_vertical_difference=1200, 
    boundary='test/data/nyc-boundary.gpkg',
    unit = 'ft',
    max_horizontal_distance=10000
)

In [4]:
svf_calculated = rioxarray.open_rasterio(svf_file)

In [3]:
#calculo da amostra para 95% de confiança com 5% de margem de erro para
822920000/0.3048*1.25
# 385 amostras
# https://pt.surveymonkey.com/mp/sample-size-calculator/

3374835958.005249

In [6]:
gdf_points = gpd.read_file('validation/nyc_validation.gpkg')

In [7]:
t = svf.xmds.rio.transform()
t

Affine(1.25, 0.0, 912500.0,
       0.0, -1.25, 275000.0)

In [8]:
points = \
gdf_points.geometry.apply(
    lambda x: (int((x.x - t[2]) / t[0]), int((x.y - t[5]) / t[4]))
)

In [9]:
60000/1.25

48000.0

In [10]:
#np.indices(svf.xmds[0, -0:int(30000/1.25), 0:int(30000/1.25)].shape)

In [11]:
svf.horizontal_distance

41000.0

In [12]:
svfs, svfs_calculated = [], []

In [13]:
int(svf_calculated[0,88887,63596 ].values)

202

In [14]:
points

0      (114598, 68099)
1       (87355, 51061)
2       (97112, 22052)
3       (86186, 72544)
4      (107538, 65019)
            ...       
380     (85653, 33674)
381     (7393, 121055)
382    (109027, 60448)
383     (99408, 27085)
384    (102478, 50803)
Name: geometry, Length: 385, dtype: object

In [15]:
for i, p in enumerate(points):
    print(f'{i}/{len(points)}', end=' ')
    print(p, end=" - ")
    col, row = p
    s = svf.calc_svf_point(col=col, row=row)
    sc = svf_calculated[0, row, col]
    svfs.append(s)
    svfs_calculated.append(int(sc.values))
    print(f'{s} - {sc} - {sc/256}')

0/385 (114598, 68099) - 0.7737347499383433 - 234.0 - 0.9140625
1/385 (87355, 51061) - 0.8713171203194459 - 196.0 - 0.765625
2/385 (97112, 22052) - 0.7986687600309792 - 186.0 - 0.7265625
3/385 (86186, 72544) - 0.521742585872301 - 157.0 - 0.61328125
4/385 (107538, 65019) - 0.33164177960272995 - 142.0 - 0.5546875
5/385 (78967, 40589) - 0.9374367666879901 - 257.0 - 1.00390625
6/385 (98701, 56783) - 0.8564583273170364 - 193.0 - 0.75390625
7/385 (92965, 41379) - 0.5927038160010429 - 178.0 - 0.6953125
8/385 (107260, 56596) - 0.8973272604345897 - 243.0 - 0.94921875
9/385 (93453, 53708) - 0.9203125414580359 - 253.0 - 0.98828125
10/385 (71708, 50077) - 0.05463747485521028 - 42.0 - 0.1640625
11/385 (54233, 58518) - 0.8193613631239182 - 222.0 - 0.8671875
12/385 (29707, 83404) - 0.7229474302400964 - 111.0 - 0.43359375
13/385 (23810, 93091) - 0.7266566686718008 - 229.0 - 0.89453125
14/385 (99701, 16851) - 0.9482838630227286 - 258.0 - 1.0078125
15/385 (72865, 82221) - 0.6727896539440475 - 218.0 - 0.8

In [20]:
gdf_points.loc[:, 'svf'] = pd.Series(svfs)

In [21]:
gdf_points.loc[:, 'svf_calculated'] = pd.Series(svfs_calculated)

In [22]:
gdf_points.loc[:, 'svf_calculated_perc'] = pd.Series(svfs_calculated)/256

In [23]:
gdf_points.to_file("validation/nyc_patial_validation.gpkg", driver='GPKG')

In [25]:
gdf_points.corr()

,rand_point_id,svf_calculated,svf_calculated_perc,svf
rand_point_id,1.000000,-0.126786,-0.126786,-0.141143
svf_calculated,-0.126786,1.000000,1.000000,0.918454
svf_calculated_perc,-0.126786,1.000000,1.000000,0.918454
svf,-0.141143,0.918454,0.918454,1.000000
